In [21]:
import os
import sys
import yaml
import json
import torch_directml
import shutil
import xmltodict
import numpy as np
import pandas as pd
import ultralytics
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

print("Torch DirectML:", torch_directml.is_available())
print("GPU name:", torch_directml.device_name(0))

Torch DirectML: True
GPU name: Intel(R) Arc(TM) Graphics 


In [22]:
input_dir = 'annotated_outdoor_yolo'
output_dir = 'dataset_yolo'

os.makedirs(os.path.join(output_dir), exist_ok=True)

images_dir = os.path.join(input_dir, 'images')
labels_dir = os.path.join(input_dir, 'labels')

In [31]:
# Get the list of image files (assuming all formats are .jpg, change if needed)
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith(".jpg")])

# Create dataset indices
dataset_size = len(image_files)
indices = list(range(dataset_size))

# Split into train (70%), val (10%), test (20%)
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_indices, test_size=0.125, random_state=42)

# Store indices in a dictionary
split_data = {
    "train": train_indices,
    "val": val_indices,
    "test": test_indices
}

print(f"Train set size: {len(train_indices)} - {len(train_indices)/len(indices)*100:.2f}%")
print(f"Validation set size: {len(val_indices)} - {len(val_indices)/len(indices)*100:.2f}%")
print(f"Test set size: {len(test_indices)} - {len(test_indices)/len(indices)*100:.2f}%\n")

# Save the split indices as JSON
json_split_path = os.path.join(output_dir, "dataset_split.json")
with open(json_split_path, "w") as f:
    json.dump(split_data, f, indent=4)

print(f"Dataset split saved to {json_split_path}")

Train set size: 217 - 69.77%
Validation set size: 31 - 9.97%
Test set size: 63 - 20.26%

Dataset split saved to dataset_yolo\dataset_split.json


In [ ]:
# List and sort both image and label files
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.txt')])

# Sanity check
assert len(image_files) == len(label_files), "Image and label counts do not match!"

# Define dataset splits
splits = ["train", "val", "test"]

# Create output directories
for split in splits:
    for category in ["images", "labels"]:
        os.makedirs(os.path.join(output_dir, split, category), exist_ok=True)

# Loop over your splits
for split, indices in split_data.items():
    for idx in indices:
        image_name = image_files[idx]
        label_name = label_files[idx]

        image_src = os.path.join(images_dir, image_name)
        label_src = os.path.join(labels_dir, label_name)

        image_dest = os.path.join(output_dir, split, "images", image_name)
        label_dest = os.path.join(output_dir, split, "labels", label_name)

        shutil.copy(image_src, image_dest)
        shutil.copy(label_src, label_dest)

# Print final counts for verification
print("\n✅ Copy complete! Summary:\n")
for split in split_data.keys():
    image_folder = os.path.join(output_dir, split, "images")
    label_folder = os.path.join(output_dir, split, "labels")

    num_images = len([f for f in os.listdir(image_folder) if f.endswith('.jpg')]) if os.path.exists(image_folder) else 0
    num_labels = len([f for f in os.listdir(label_folder) if f.endswith('.txt')]) if os.path.exists(label_folder) else 0

    print(f"Split '{split}': {num_images} images, {num_labels} labels.")


✅ Copy complete! Summary:

Split 'train': 284 images, 284 labels.
Split 'val': 59 images, 59 labels.
Split 'test': 115 images, 115 labels.
